In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from pathlib import Path

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

In [2]:
api = SentinelAPI(os.environ.get('sentinelAPI.user'), os.environ.get('sentinelAPI.pass'), 'https://scihub.copernicus.eu/dhus')

In [3]:
# search by polygon, time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson(r'../inData/diskoIsland.geojson'))

def findS1Data(fp=footprint,prodType='SLC',ron=171,sliceNo=2):

    products = api.query(fp,
                         platformname='Sentinel-1',
                         producttype=prodType,
                         relativeOrbitNumber=ron,
                         sliceNumber=sliceNo
                         )
    print("Number of products found: "+str(len(products)))
    return products

In [4]:
pSLC = findS1Data()

Number of products found: 29


In [5]:
p1 = list(pSLC.items())[2][1]

In [6]:
p1

{'title': 'S1B_IW_SLC__1SDH_20200218T101603_20200218T101631_020322_0267D2_1C84',
 'link': "https://scihub.copernicus.eu/dhus/odata/v1/Products('2a54553a-2ee4-4c5e-ba05-f3b713b788d2')/$value",
 'link_alternative': "https://scihub.copernicus.eu/dhus/odata/v1/Products('2a54553a-2ee4-4c5e-ba05-f3b713b788d2')/",
 'link_icon': "https://scihub.copernicus.eu/dhus/odata/v1/Products('2a54553a-2ee4-4c5e-ba05-f3b713b788d2')/Products('Quicklook')/$value",
 'summary': 'Date: 2020-02-18T10:16:03.576Z, Instrument: SAR-C SAR, Mode: HH HV, Satellite: Sentinel-1, Size: 7.68 GB',
 'beginposition': datetime.datetime(2020, 2, 18, 10, 16, 3, 576000),
 'endposition': datetime.datetime(2020, 2, 18, 10, 16, 31, 501000),
 'ingestiondate': datetime.datetime(2020, 2, 18, 14, 18, 21, 248000),
 'missiondatatakeid': 157650,
 'slicenumber': 2,
 'orbitnumber': 20322,
 'lastorbitnumber': 20322,
 'relativeorbitnumber': 171,
 'lastrelativeorbitnumber': 171,
 'sensoroperationalmode': 'IW',
 'swathidentifier': 'IW1 IW2 IW3'

In [7]:
# check how many products are online
def checkOnlineProds(products):
    onlineProds = 0
    products_online = []
    online_dates = []
    for p in products:
        #print(api.get_product_odata(p))
        product_info = api.get_product_odata(p)
        #print (product_info)
        # extract September data
        #if product_info['date'].month == 9:
        if product_info['Online']:
            onlineProds +=1
            products_online.append(product_info)
            if product_info['date'] not in online_dates:
                online_dates.append(product_info['date'])
    print('Number of products online:', onlineProds)
                    
    return products_online, online_dates

In [8]:
pSLC_online, pOnline_dates = checkOnlineProds(pSLC)

Number of products online: 29


In [9]:
pOnline_dates

[datetime.datetime(2020, 3, 13, 10, 16, 3, 478000),
 datetime.datetime(2020, 3, 1, 10, 16, 3, 290000),
 datetime.datetime(2020, 2, 18, 10, 16, 3, 576000),
 datetime.datetime(2020, 2, 6, 10, 16, 3, 874000),
 datetime.datetime(2020, 1, 25, 10, 16, 4, 139000),
 datetime.datetime(2020, 1, 13, 10, 16, 4, 460000),
 datetime.datetime(2020, 1, 1, 10, 16, 4, 950000),
 datetime.datetime(2019, 12, 20, 10, 16, 5, 599000),
 datetime.datetime(2019, 12, 8, 10, 16, 6, 89000),
 datetime.datetime(2019, 11, 26, 10, 16, 6, 483000),
 datetime.datetime(2019, 11, 14, 10, 16, 6, 878000),
 datetime.datetime(2019, 11, 2, 10, 16, 6, 775000),
 datetime.datetime(2019, 10, 21, 10, 16, 7, 4000),
 datetime.datetime(2019, 10, 9, 10, 16, 6, 780000),
 datetime.datetime(2019, 9, 27, 10, 16, 6, 952000),
 datetime.datetime(2019, 9, 15, 10, 16, 6, 300000),
 datetime.datetime(2019, 9, 3, 10, 16, 5, 747000),
 datetime.datetime(2019, 8, 22, 10, 16, 5, 160000),
 datetime.datetime(2019, 8, 10, 10, 16, 4, 557000),
 datetime.datet

In [ ]:
#downloadDir = './sepDataGRD'
#Path(downloadDir).mkdir(parents=True, exist_ok=True)

In [ ]:
downloadDir = './SLCData'
Path(downloadDir).mkdir(parents=True, exist_ok=True)

In [ ]:
# download products for Sep that are online
for op in pSLC_online:
    api.download(op['id'], downloadDir)

In [ ]:
#Download all products 
#api.download_all(products)